In [ ]:
include(Pkg.dir("HybridSystems", "examples", "cruise_control.jl"));

In [ ]:
const va = 15.6
#const vb = 24.5
#const vc = 29.5
const v = (va, )
const U = 4.
const h = 0.4
const M = 1
const d = 4
const m0 = 500

In [ ]:
using MathOptInterfaceMosek
sdpsolver = MosekOptimizer(LOG=0);

In [ ]:
using CSDP
sdpsolver = CSDPOptimizer(printlevel=0);

In [ ]:
using SwitchOnSafety
const SOS = SwitchOnSafety;

In [ ]:
using LightGraphs
const MOI = MathOptInterface
function fullsolve(T, M, H, oneshot = false, onlyone = false, detcone=MOI.RootDetConeTriangle)
    N = 1 + (T+1) * length(v)
    hs = cruise_control_example(N, M, vmin = 5., v=v, U=U, H=H, T=T, sym=false, m0 = m0)
    function hv(v)
        h = zeros(statedim(hs, 1))
        for i in 1:M
            h[2i] = (5.+v)/2
        end
        h[2M+1] = (5.+v)/2
        h
    end
    habc = SOS.InteriorPoint.(hv.(v))
    ha = habc[1]
    hi = fill(ha, N)
    if oneshot
        λ = Dict(Edge(a, b) => 1. for a in 1:N, b in 1:N)
    else
        λ = Dict(Edge(a, a) => 1. for a in 1:N)
    end
    if oneshot
        getis(hs, sdpsolver, hi, λ = λ, detcone=detcone);
    else
        p = Vector{SwitchOnSafety.Ellipsoid{Float64}}(N)
        for i in (onlyone ? 1:1 : 1:N)
            fillis!(p, i:i, hs, sdpsolver, hi, λ = λ, enabled=1:i, detcone=detcone)
            @show inv(det(p[i].Q))
        end
        p
    end
end

In [ ]:
import Plots
Plots.pyplot()
_savefig(name) = Plots.savefig("/home/blegat/Dropbox/Research/Images/CruiseControl$name.eps");

In [ ]:
using JuMP
const MOI = MathOptInterface

In [ ]:
function elltosoc(p::SOS.Ellipsoid)
    U, S, V = svd(p.Q)
    L = diagm(sqrt.(S)) * V'
    @show p.c
    x -> L * (x - p.c)
end

In [ ]:
using JuMP
const MOI = MathOptInterface

function mpcstep(hs, modes, x0, y0, soc, usetol)
    m = Model()
    nt = length(modes)
    y = @variable m [1:nt]
    x = @variable m [1:nt, 1:d]
    u = @variable m [1:(nt-1)]
    @constraint m x[1, :] .== x0
    @constraint m y[1] == y0
    if usetol
        tol = @variable m lowerbound=0
    end
    for i in 1:nt
        q = modes[i]
        @assert !Polyhedra.hashyperplanes(stateset(hs, q))
        for h in halfspaces(stateset(hs, q))
            # h.a[end] != 0 means that it is u <= 4 or u >= -4 for which we don't want to use tolerance
            # It is a hard constraint
            # Same for h.a[1] != 0
            # @constraint m dot(x[i, :], h.a) <= h.β + (usetol && iszero(h.a[1]) && iszero(h.a[end])? tol : 0.)
            @constraint m dot(x[i, :], h.a) <= h.β + (usetol ? tol : 0.)
        end
        if soc !== nothing
            @constraint m [1; soc[q](x[i, :])] in MOI.SecondOrderCone(1+d)
        end
        if i < nt
            s = modes[i+1]
            σ = symbol(hs.automaton, LightGraphs.Edge(q, s))
            r = hs.resetmaps[σ]
            @constraint m x[i+1, :] .== r.A * x[i, :] + r.B .* u[i]
            if σ == 1
                @constraint m y[i+1] == y[i] + h * x[i, d-1]
            else
                @constraint m y[i+1] == y[i]
            end
        end
    end
    @objective m Max y[end] - (usetol ? 1000tol : 0.)
    MOI.empty!(sdpsolver)
    MOI.Utilities.resetoptimizer!(m, sdpsolver)
    JuMP.optimize(m)

    @assert JuMP.terminationstatus(m) == MOI.Success
    JuMP.dualstatus(m), JuMP.resultvalue.(x)[2:end,:], JuMP.resultvalue.(y)[2:end]
end

In [ ]:
function solvempc(nt, nsteps, useellipsoids = false)
    T = nsteps
    na = div(nt,2)-1
    @show nt
    @show na
    @show T
    nN = nt-T-na
    @show nN
    N = 1 + (T+1) * length(v)
    startstate = N
    modes = [fill(startstate, nN); collect((T+1):-1:2); fill(1, na)]
    @assert length(modes) == nt
    y = Vector{Float64}(nt)
    x = Matrix{Float64}(nt, d)
    x[1, :] = [repmat([0., 10.], M); 10.; 0.]
    y[1] = -10. * h
    last = nt
    if useellipsoids
        soc = elltosoc.(fullsolve(T, M, h*T, false, false))
    else
        soc = nothing
    end
    hs = cruise_control_example(N, M, vmin = 5., v=v, U=U, H=h*T, T=T, sym=false, m0 = m0)
    for i in 1:(nt-1)
        j = min(nt, i + nsteps)
        _step(usetol) = mpcstep(hs, modes[i:j], x[i, :], y[i], soc, usetol)
        ds, X, Y = _step(false)
        if ds == MOI.InfeasibilityCertificate
            ds, X, Y = _step(true)
        end
        while ds == MOI.InfeasibilityCertificate && nsteps > 1
            nsteps -= 1
            ds, X, Y = _step(true)
        end
        x[i+1, :] = X[1, :]
        y[i+1] = Y[1]
        if last == nt && (ds == MOI.InfeasibilityCertificate || !(X[1, :] in stateset(hs, modes[i+1])))
            @show X[1, :]
            last = i+1
            #break
        end
    end
    if last < nt
        warn("$last < $nt")
    end
    take = last
    #take = nt
    if take > nN
        @show y[nN]
        @show y[nN+1]
        @show y[nN+2]
        [x[2:nN, :]; x[(nN+2):take, :]], [y[2:nN]; y[(nN+2):take]]
    else
        x[2:take, :], y[2:take]
    end
end

In [ ]:
#nt = 77
nt = 152
tswitch = [30., 30.]
t = [0.]
for i in 2:(nt-2)
    push!(t, t[end] + h)
end
minsafe = 24

In [ ]:
XYok = Dict{String, Tuple{Matrix{Float64}, Vector{Float64}}}()
XYko = Dict{String, Tuple{Matrix{Float64}, Vector{Float64}}}();

In [ ]:
#XYok["unsafe optimal"] = solvempc(nt, nt - div(nt,2));

In [ ]:
#XYok["unsafe $minsafe"] = solvempc(nt, minsafe);

In [ ]:
XYok["unsafe $(minsafe-1)"] = solvempc(nt, minsafe-1);

In [ ]:
length(XYok["unsafe $(minsafe-1)"][2])

In [ ]:
#XYko["unsafe 8"] = solvempc(nt, 8);

In [ ]:
#XYok["safe optimal"] = solvempc(nt, div(nt,2), true);

In [ ]:
XYok["safe $(minsafe-1)"] = solvempc(nt, minsafe-1, true);

In [ ]:
XYok["safe 3"] = solvempc(nt, 3, true);

In [ ]:
#XY = [("unsafe 8", XYko["unsafe 8"])];
XY = XYok;
plotsize = (800, 400);
# colors taken from https://previews.123rf.com/images/capacitorphoto/capacitorphoto1410/capacitorphoto141000191/32438941-Graph-Icon-color-set-illustration--Stock-Vector.jpg
red = Plots.RGBA(((0xf5, 0x92, 0x97) ./ 255)...)
gre = Plots.RGBA(((0xcb, 0xdf, 0x80) ./ 255)...)
blu = Plots.RGBA(((0x23, 0x94, 0xce) ./ 255)...)
ora = Plots.RGBA(((0xfa, 0xcb, 0x95) ./ 255)...)
yel = Plots.RGBA(((0xf2, 0xf0, 0x8b) ./ 255)...)
colors = Dict("unsafe $(minsafe-1)" => ora, "safe $(minsafe-1)" => blu, "safe 3" => gre)

In [ ]:
Plots.plot(legendfont=Plots.font(13), guidefont=Plots.font(16), tickfont=Plots.font(10), grid=false, yticks=[10, 15.6, 35], size=plotsize, xlabel="Time [s]", ylabel="Speed [m/s]")
Plots.plot!([t[1], tswitch[1]], [35, 35], linewidth=0.5, color=:black, label="")
Plots.plot!([tswitch[1], t[end]], [va, va], linewidth=0.5, color=:black, label="")
mini = Inf
maxi = 35
for (s, (x, y)) in XY
    v1 = x[:,d-2]
    v0 = x[:,d-1]
    #:none,:auto,:circle,:rect,:star5,:diamond,:hexagon,:cross,:xcross,:utriangle,:dtriangle,:rtriangle,:ltriangle,:pentagon,:heptagon,:octagon,:star4,:star6,:star7,:star8,:vline,:hline,:+,:x].
    issafe = s[1] == 's'
    ms = 7 + (issafe ? 0 : 1)
    #Plots.scatter!(t[1:end-1], v1[1:end], label="Trailer speed $s", markersize=ms, markershape=issafe ? :diamond : :utriangle, markerstrokewidth=0)
    v0 = v0[1:end-1]
    Plots.scatter!(t[1:(length(v0))], v0, color = colors[s], label="Truck speed $s", markersize=ms, markershape=issafe ? :circle : :rtriangle, markerstrokewidth=0)
    mini = min(mini, minimum(v1), minimum(v0))
    maxi = max(maxi, maximum(v1), maximum(v0))
end
#Plots.plot!(tswitch, [mini, maxi])
#Plots.plot!(tswitch, [mini, maxi], linewidth=2, label="")
if length(XY) == 1
    _savefig("UnsafeSpeed")
else
    @assert length(XY) == 3
    _savefig("Speed")
end
Plots.plot!()

In [ ]:
Plots.plot(legendfont=Plots.font(13), guidefont=Plots.font(16), tickfont=Plots.font(10), size=plotsize, grid=false, ylim=(-4.3, 4.3), xlabel="Time [s]", ylabel="Acceleration [m/s²]")
Plots.plot!([0, 60], [-4, -4], linewidth=0.5, color=:black, label="")
Plots.plot!([0, 60], [4, 4], linewidth=0.5, color=:black, label="")
for (s, (x, y)) in XY
    u = x[:, 4]
    issafe = s[1] == 's'
    u = u[1:end-2]
    ms = 7 + (issafe ? 0 : 1)
    if !issafe
        # Choose from :none,:auto,:circle,:rect,:star5,:diamond,:hexagon,:cross,:xcross,:utriangle,:dtriangle,:rtriangle,:ltriangle,:pentagon,:heptagon,:octagon,:star4,:star6,:star7,:star8,:vline,:hline,:+,:x].

        Plots.scatter!([t[length(u)]], [u[end]], markersize=ms+5, markerstrokewidth=3, markershape=:circle, markercolor=:white, markerstrokecolor = red, label="Constraint violation for $s")
    end
    Plots.scatter!(t[1:length(u)], u, label="Truck acceleration $s", color = colors[s], markersize=ms, markershape=issafe ? :circle : :rtriangle, markerstrokewidth=0)
    Plots.scatter!()
end
#Plots.plot!(tswitch, [-U, U], label="")
if length(XY) == 1
    _savefig("UnsafeAcceleration")
else
    @assert length(XY) == 3
    _savefig("Acceleration")
end
Plots.plot!()

In [ ]:
Plots.plot(size=(800, 400), xlabel="Time [s]", ylabel="Truck position [m]")
for (s, (x, y)) in XY
    Plots.plot!(t, y, label=s)
end
Plots.plot!(tswitch, collect(extrema(XY["unsafe optimal"][2])), label="")
#_savefig("Position")

In [ ]:
Plots.plot(yticks = [-0.5, -0.25, 0.0, 0.25, 0.5], ylims=(-0.55, 0.55), size=plotsize, xlabel="Time [s]", ylabel="Displacement [m]")
for (s, (x, y)) in XY
    dist = x[:, 1]
    Plots.plot!(t[1:length(dist)], dist, label="Spring displacement $s")
end
Plots.plot!()
#Plots.plot!(tswitch, [-.5, .5], label="")
#_savefig("Displacement")